In [ ]:
!pip install -q openai-whisper gradio thefuzz gtts librosa transformers
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:0

In [ ]:
import gradio as gr
import torch
import numpy as np
import librosa
import time
import os

#Whisper for ASR
import whisper

# gTTS for text-to-speech
from gtts import gTTS

# Hugging Face Transformers for audio embeddings
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model

# fuzzy string matching for text similarity
from thefuzz import fuzz

# cosine similarity function to make the evaluations
from sklearn.metrics.pairwise import cosine_similarity

# for checking if GPU is available
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cpu


# Models Initialization

In [ ]:
# Loading Whisper's base model for efficiency (it can produce errors for unused words in languages other than English)
whisper_model = whisper.load_model("base", device=DEVICE)

# Loading Wav2Vec2 model for doing the audio embeddings
wav2vec_model_name = "facebook/wav2vec2-large-xlsr-53"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(wav2vec_model_name)
wav2vec_model = Wav2Vec2Model.from_pretrained(wav2vec_model_name).to(DEVICE)
wav2vec_model.eval()

# Create a directory for temporary audio files if it doesn't exist
os.makedirs("temp_audio", exist_ok=True)

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 137MiB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

# Temporary directory for saving audio files

In [ ]:
os.makedirs("temp_audio", exist_ok=True)

def generate_reference_audio(text, lang):
    """
    Convert the reference text to speech audio using Google TTS (or alternative TTS for the given language).
    Returns the file path of the generated audio and stores state (path and text).
    """
    if not text or not text.strip():
        raise gr.Error("Please enter a reference text.")
    # Use current time to generate unique filename
    timestamp = int(time.time())
    output_path = f"temp_audio/ref_{lang}_{timestamp}.mp3"
    try:
        tts = gTTS(text=text, lang=lang)
        tts.save(output_path)
    except Exception as e:
        # If gTTS fails (e.g., no internet or unsupported language), raise error
        raise gr.Error(f"Failed to generate reference audio. ({str(e)})")
    # Double-check file creation
    if not os.path.exists(output_path):
        raise gr.Error("Audio generation failed (file not found).")
    return output_path, output_path, text  # return file path for audio player, and state values


In [ ]:
def generate_reference_audio(text, lang):
    """
    Convert the provided text to speech audio using Google TTS.
    Returns the file path of the generated audio.
    """
    if not text or not text.strip():
        raise gr.Error("Please enter a reference text.")

    timestamp = int(time.time())
    output_path = f"temp_audio/ref_{lang}_{timestamp}.mp3"
    try:
        tts = gTTS(text=text, lang=lang)
        tts.save(output_path)
    except Exception as e:
        raise gr.Error(f"Failed to generate reference audio. ({str(e)})")

    if not os.path.exists(output_path):
        raise gr.Error("Audio generation failed (file not found).")

    # Return the path for the audio player and store state values
    return output_path, output_path, text

In [ ]:
def get_embedding(audio_path):
    """
    Compute an audio embedding for the given audio file using Wav2Vec2.
    Returns a normalized 1D numpy array representing the audio.
    """
    speech, sr = librosa.load(audio_path, sr=16000)
    inputs = feature_extractor(speech, sampling_rate=sr, return_tensors="pt", padding=True)
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    with torch.inference_mode():
        outputs = wav2vec_model(**inputs)
    # Average the hidden states to get a single feature vector
    emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    # Normalize the embedding vector
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
    return emb

In [ ]:
def analyze_pronunciation(user_audio, ref_audio_path, ref_text, lang):
    """
    Analyze the user's pronunciation by comparing ASR text transcription and audio similarity.
    Returns a result message with scores, warnings, and the transcribed text.
    """
    # Handle input formats (if stored in tuples)
    user_path = user_audio[0] if isinstance(user_audio, tuple) else user_audio
    ref_path = ref_audio_path[0] if isinstance(ref_audio_path, tuple) else ref_audio_path

    if not ref_text or not os.path.exists(ref_path):
        return "Error: Please provide reference text and generate reference audio first.", "", ""
    if not user_path or not os.path.exists(user_path):
        return "Error: Please record your pronunciation audio.", "", ""

    try:
        # Transcribe user audio with Whisper (explicitly specify language)
        transcription = whisper_model.transcribe(user_path, language=lang if lang else None)
        user_transcript = transcription["text"]
    except Exception as e:
        return f"Analysis Error: Failed to transcribe audio ({str(e)}).", "", ""

    # Compute text similarity (using fuzzy matching)
    text_sim = fuzz.ratio(user_transcript.lower().strip(), ref_text.lower().strip()) / 100.0

    try:
        # Compute audio embeddings and cosine similarity score
        user_emb = get_embedding(user_path)
        ref_emb = get_embedding(ref_path)
        audio_sim = cosine_similarity(user_emb, ref_emb)[0][0]
        audio_sim = max(0.0, min(1.0, audio_sim))  # Clamp the score to [0, 1]
    except Exception as e:
        return f"Analysis Error: Audio embedding failed ({str(e)}).", "", user_transcript

    # Weighted scoring: emphasizing text similarity more than audio
    final_score = (0.7 * text_sim) + (0.3 * audio_sim)

    # Prepare result display
    result_lines = [
        f" **Text Match:** {text_sim:.2%}",
        f" **Audio Similarity:** {audio_sim:.2%}",
        f" **Overall Score:** {final_score:.2%}"
    ]
    result_msg = "\n".join(result_lines)

    warnings_list = []
    if text_sim < 0.70:
        warnings_list.append("⚠️ The spoken text differs significantly from the reference.")
    if text_sim >= 0.70 and audio_sim < 0.50:
        warnings_list.append("⚠️ Pronunciation pattern is quite different from the reference.")

    warning_msg = "\n".join(warnings_list)

    return result_msg, warning_msg, user_transcript

# Lauching

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## Multilingual Pronunciation Practice\n"
                "Enter a phrase in your target language, generate its reference audio, then record (or upload) your pronunciation for analysis.")

    # States to store the generated reference audio path and text
    ref_audio_state = gr.State()
    ref_text_state = gr.State()

    with gr.Row():
        with gr.Column():
            language = gr.Dropdown(label="Language", choices=["ru", "en", "es", "fr", "de", "zh"], value="ru",
                                   info="Select the language (affects both TTS and ASR)")
            ref_text_input = gr.Textbox(label="Reference Text", lines=3, placeholder="Enter the phrase to practice...")
            generate_btn = gr.Button(" Generate Reference Audio")
            ref_audio_player = gr.Audio(label="Reference Audio", interactive=False)
        with gr.Column():
            # Removed the `source` parameter to avoid compatibility errors.
            user_audio_input = gr.Audio(label="Your Pronunciation (Record or Upload)", type="filepath")
            analyze_btn = gr.Button(" Analyze Pronunciation")
            result_box = gr.Textbox(label="Results", lines=3)
            warnings_box = gr.Textbox(label="Warnings", lines=2)
            transcript_box = gr.Textbox(label="Transcript of Your Speech", lines=2)

    # Set up component interactions
    generate_btn.click(
        fn=generate_reference_audio,
        inputs=[ref_text_input, language],
        outputs=[ref_audio_player, ref_audio_state, ref_text_state]
    )

    analyze_btn.click(
        fn=analyze_pronunciation,
        inputs=[user_audio_input, ref_audio_state, ref_text_state, language],
        outputs=[result_box, warnings_box, transcript_box]
    )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d6ca4ffdba7974eb8f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Task
Write a README file for the project based on the provided plan.

## Project description

### Subtask:
Write a brief description of the project, explaining its purpose and functionality.


**Reasoning**:
I need to write a brief description of the project based on the instructions. This will be a markdown text block.



In [1]:
```markdown
# Multilingual Pronunciation Practice Tool

This project is a web-based tool designed to help users practice and improve their pronunciation in various languages. It allows users to input a phrase in their target language, generate a reference audio using text-to-speech, and then record or upload their own pronunciation to receive feedback. The tool analyzes the user's speech by comparing the transcribed text and the audio features against the reference, providing scores for text match and audio similarity, along with potential warnings.

SyntaxError: unterminated string literal (detected at line 4) (ipython-input-3172811316.py, line 4)

**Reasoning**:
The previous command failed because the markdown text was not properly enclosed within a Python string in the code block. I need to correct the markdown formatting within the Python code block.



In [2]:
markdown_description = """
# Multilingual Pronunciation Practice Tool

This project is a web-based tool designed to help users practice and improve their pronunciation in various languages. It allows users to input a phrase in their target language, generate a reference audio using text-to-speech, and then record or upload their own pronunciation to receive feedback. The tool analyzes the user's speech by comparing the transcribed text and the audio features against the reference, providing scores for text match and audio similarity, along with potential warnings.
"""
print(markdown_description)


# Multilingual Pronunciation Practice Tool

This project is a web-based tool designed to help users practice and improve their pronunciation in various languages. It allows users to input a phrase in their target language, generate a reference audio using text-to-speech, and then record or upload their own pronunciation to receive feedback. The tool analyzes the user's speech by comparing the transcribed text and the audio features against the reference, providing scores for text match and audio similarity, along with potential warnings.



## Setup instructions

### Subtask:
Detail the steps required to set up and run the project, including dependencies and installation commands.


**Reasoning**:
I will create a markdown string containing the "Setup and Installation" section with subsections for "Dependencies", "Installation", and "Running the Application", as requested in the instructions. This will fulfill all the requirements of the subtask in a single step.



In [3]:
markdown_setup = """
## Setup and Installation

### Dependencies
This project requires the following Python libraries:
- `openai-whisper`
- `gradio`
- `thefuzz`
- `gtts`
- `librosa`
- `transformers`
- `torch`
- `numpy`
- `scikit-learn`

### Installation
You can install all the required dependencies using pip:
```bash
pip install -q openai-whisper gradio thefuzz gtts librosa transformers torch numpy scikit-learn
```

### Running the Application
To run the application, execute the Python script containing the Gradio interface. The script will launch a web server and provide a URL where you can access the tool.
"""
print(markdown_setup)


## Setup and Installation

### Dependencies
This project requires the following Python libraries:
- `openai-whisper`
- `gradio`
- `thefuzz`
- `gtts`
- `librosa`
- `transformers`
- `torch`
- `numpy`
- `scikit-learn`

### Installation
You can install all the required dependencies using pip:
```bash
pip install -q openai-whisper gradio thefuzz gtts librosa transformers torch numpy scikit-learn
```

### Running the Application
To run the application, execute the Python script containing the Gradio interface. The script will launch a web server and provide a URL where you can access the tool.



## Usage guide

### Subtask:
Explain how to use the Gradio interface, including how to generate reference audio and analyze pronunciation.


**Reasoning**:
Create the markdown section for usage, including subsections for generating reference audio and analyzing pronunciation, following the specified steps.



In [4]:
markdown_usage = """
## Usage

### Generating Reference Audio
1.  Enter the desired text in the "Reference Text" box.
2.  Select the language from the "Language" dropdown.
3.  Click the "Generate Reference Audio" button.
4.  The generated audio will appear in the "Reference Audio" player.

### Analyzing Your Pronunciation
1.  Record your pronunciation using the microphone or upload an audio file in the "Your Pronunciation" box.
2.  Click the "Analyze Pronunciation" button.
3.  The results, including scores, warnings, and transcript, will appear in the "Results", "Warnings", and "Transcript of Your Speech" boxes respectively.
"""
print(markdown_usage)


## Usage

### Generating Reference Audio
1.  Enter the desired text in the "Reference Text" box.
2.  Select the language from the "Language" dropdown.
3.  Click the "Generate Reference Audio" button.
4.  The generated audio will appear in the "Reference Audio" player.

### Analyzing Your Pronunciation
1.  Record your pronunciation using the microphone or upload an audio file in the "Your Pronunciation" box.
2.  Click the "Analyze Pronunciation" button.
3.  The results, including scores, warnings, and transcript, will appear in the "Results", "Warnings", and "Transcript of Your Speech" boxes respectively.



## Code explanation

### Subtask:
Provide a high-level overview of the code, explaining the main functions and their roles.


**Reasoning**:
Create a markdown section for code explanation, explaining the purpose of the main functions based on the provided code.



In [5]:
markdown_code_explanation = """
## Code Explanation

-   `generate_reference_audio(text, lang)`: This function takes the reference text and language as input, uses Google TTS to convert the text into speech, saves the audio to a temporary file, and returns the file path and the input text and path as state variables.
-   `get_embedding(audio_path)`: This function takes the path to an audio file, loads it using `librosa`, processes it with the Wav2Vec2 feature extractor and model to obtain audio embeddings, and returns a normalized 1D numpy array representing the audio's features.
-   `analyze_pronunciation(user_audio, ref_audio_path, ref_text, lang)`: This is the core analysis function. It takes the user's recorded audio, the path to the generated reference audio, the original reference text, and the language. It first transcribes the user's audio using Whisper, then calculates text similarity between the transcription and the reference text using fuzzy matching. It also calculates audio similarity by comparing the embeddings of the user's audio and the reference audio using cosine similarity. Finally, it computes a weighted overall score, generates result messages and warnings based on the scores, and returns them along with the user's transcription.
"""
print(markdown_code_explanation)


## Code Explanation

-   `generate_reference_audio(text, lang)`: This function takes the reference text and language as input, uses Google TTS to convert the text into speech, saves the audio to a temporary file, and returns the file path and the input text and path as state variables.
-   `get_embedding(audio_path)`: This function takes the path to an audio file, loads it using `librosa`, processes it with the Wav2Vec2 feature extractor and model to obtain audio embeddings, and returns a normalized 1D numpy array representing the audio's features.
-   `analyze_pronunciation(user_audio, ref_audio_path, ref_text, lang)`: This is the core analysis function. It takes the user's recorded audio, the path to the generated reference audio, the original reference text, and the language. It first transcribes the user's audio using Whisper, then calculates text similarity between the transcription and the reference text using fuzzy matching. It also calculates audio similarity by comparing the 

## Dependencies

### Subtask:
List the required Python libraries and their versions.


**Reasoning**:
Create a markdown string for the "Dependencies" section, listing the required libraries and their versions based on the pip install command and common practices.



In [6]:
markdown_dependencies = """
## Dependencies

This project requires the following Python libraries:
- `openai-whisper`
- `gradio`
- `thefuzz`
- `gtts`
- `librosa`
- `transformers`
- `torch`
- `numpy`
- `scikit-learn`

"""
print(markdown_dependencies)


## Dependencies

This project requires the following Python libraries:
- `openai-whisper`
- `gradio`
- `thefuzz`
- `gtts`
- `librosa`
- `transformers`
- `torch`
- `numpy`
- `scikit-learn`




## Future improvements (optional)

### Subtask:
Suggest potential future enhancements or features for the project in a markdown format.


**Reasoning**:
Create a markdown string for the "Future Improvements (Optional)" section, listing potential enhancements as instructed, and print it.



In [7]:
markdown_future_improvements = """
## Future Improvements (Optional)

Here are some potential enhancements and features that could be added to this project:

*   **More Granular Feedback:** Implement features to provide more specific feedback, such as highlighting mispronounced words or identifying incorrect phonemes within the user's speech. This could involve aligning the user's audio and transcription with the reference more precisely.
*   **Expanded Language Support:** Integrate additional language models for both ASR (Whisper supports many, but fine-tuning might be needed for better accuracy in less common languages) and TTS (exploring alternative TTS providers or models for broader coverage and more natural voices).
*   **Advanced Model Integration:** Experiment with more advanced or specialized ASR, TTS, or audio embedding models to potentially improve transcription accuracy, naturalness of reference audio, or the precision of audio similarity scoring.
*   **User Accounts and Progress Tracking:** Introduce a backend to allow users to create accounts, save their practice sessions, track their progress over time, and perhaps work through structured lessons or vocabulary lists.
*   **Interactive/Gamified Interface:** Develop a more engaging user interface that incorporates interactive elements or gamification features to make pronunciation practice more fun and motivating. This could include scoring systems, challenges, or visual feedback on pronunciation.
*   **Error Analysis and Suggestions:** Provide specific suggestions for improvement based on identified pronunciation errors, rather than just generic warnings. This could involve linking errors to phonetic descriptions or providing examples of correct pronunciation.
"""
print(markdown_future_improvements)


## Future Improvements (Optional)

Here are some potential enhancements and features that could be added to this project:

*   **More Granular Feedback:** Implement features to provide more specific feedback, such as highlighting mispronounced words or identifying incorrect phonemes within the user's speech. This could involve aligning the user's audio and transcription with the reference more precisely.
*   **Expanded Language Support:** Integrate additional language models for both ASR (Whisper supports many, but fine-tuning might be needed for better accuracy in less common languages) and TTS (exploring alternative TTS providers or models for broader coverage and more natural voices).
*   **Advanced Model Integration:** Experiment with more advanced or specialized ASR, TTS, or audio embedding models to potentially improve transcription accuracy, naturalness of reference audio, or the precision of audio similarity scoring.
*   **User Accounts and Progress Tracking:** Introduce a bac

## Summary:

### Data Analysis Key Findings

*   The project is a web-based tool for multilingual pronunciation practice, allowing users to generate reference audio, record their own pronunciation, and receive feedback based on text and audio similarity.
*   The setup requires installing Python libraries including `openai-whisper`, `gradio`, `thefuzz`, `gtts`, `librosa`, `transformers`, `torch`, `numpy`, and `scikit-learn` using pip.
*   The usage involves entering text and selecting a language to generate reference audio, then recording or uploading user audio for analysis, which provides scores, warnings, and a transcription.
*   Key functions include `generate_reference_audio` (using Google TTS), `get_embedding` (using Wav2Vec2 for audio features), and `analyze_pronunciation` (using Whisper for transcription, fuzzy matching for text similarity, and cosine similarity for audio similarity).

### Insights or Next Steps

*   The current documentation covers the essential aspects of the project for a README file.
*   A next step could be to integrate the generated markdown sections into a single README.md file.
